In [1]:
import warnings

import pandas as pd

from MLWrappers import SklearnBlackBox
from PrivacyAttacks import MiaPrivacyAttack, AloaPrivacyAttack
from MLPrivacyEvaluator import PrivacyEvaluator

warnings.simplefilter("ignore", UserWarning)

In [2]:
DS_NAME = 'gaussian'
DATA_FOLDER = f'./data/{DS_NAME}'

# we load the target black box model using our wrapper
target = SklearnBlackBox(f'./models/rf_{DS_NAME}.pkl')

# We load the data used to train, test of the model, as well as the shadow data
train_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_original_train_set.csv', skipinitialspace=True)
test_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_original_test_set.csv', skipinitialspace=True)
shadow_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_shadow_set.csv', skipinitialspace=True)

In [3]:
# We initialise the attacks, with the desired parameters for each
mia = MiaPrivacyAttack(target, n_shadow_models=5)
aloa = AloaPrivacyAttack(target, n_shadow_models=1, n_noise_samples_fit=200,
                         shadow_test_size=0.3, undersample_attack_dataset=True)
attacks = [mia, aloa]

In [4]:
# We initialise the PrivacyEvaluator object
# We pass the target model and the attacks we want to use
evaluator = PrivacyEvaluator(target, attacks)

In [5]:
# We use the fit() method to execute the attacks, starting from the shadow data
evaluator.fit(shadow_set)

Each shadow model uses 100.000 % of the data


100%|██████████| 4800/4800 [01:21<00:00, 58.59it/s]

[2.    1.    0.995 0.99  0.985 0.98  0.975 0.97  0.965 0.96  0.955 0.95
 0.945 0.94  0.935 0.93  0.925 0.92  0.915 0.91  0.905 0.9   0.895 0.89
 0.885 0.88  0.875 0.865 0.86  0.855 0.845 0.835 0.83  0.825 0.815 0.81
 0.805 0.8   0.77  0.755 0.715 0.665 0.64  0.6   0.59  0.575 0.565 0.525
 0.515 0.51  0.455 0.385 0.   ]


In [6]:
# Then we can obtain the performances using the report() method
results = evaluator.report(train_set, test_set)
print(results)


100%|██████████| 32000/32000 [08:30<00:00, 62.73it/s]


{'mia_attack': {'classification_report': {'IN': {'precision': 0.8074161534288541, 'recall': 0.988359375, 'f1-score': 0.8887718004109806, 'support': 25600}, 'OUT': {'precision': 0.55052790346908, 'recall': 0.05703125, 'f1-score': 0.1033555146538298, 'support': 6400}, 'accuracy': 0.80209375, 'macro avg': {'precision': 0.6789720284489671, 'recall': 0.5226953125, 'f1-score': 0.4960636575324052, 'support': 32000}, 'weighted avg': {'precision': 0.7560385034368993, 'recall': 0.80209375, 'f1-score': 0.7316885432595505, 'support': 32000}}}, 'aloa_attack': {'classification_report': {'IN': {'precision': 0.8002075591711024, 'recall': 0.9337109375, 'f1-score': 0.8618196895675218, 'support': 25600}, 'OUT': {'precision': 0.20291216533583842, 'recall': 0.0675, 'f1-score': 0.101301442138586, 'support': 6400}, 'accuracy': 0.76046875, 'macro avg': {'precision': 0.5015598622534704, 'recall': 0.50060546875, 'f1-score': 0.48156056585305385, 'support': 32000}, 'weighted avg': {'precision': 0.6807484804040496

In [7]:
results['mia_attack']

{'classification_report': {'IN': {'precision': 0.8074161534288541,
   'recall': 0.988359375,
   'f1-score': 0.8887718004109806,
   'support': 25600},
  'OUT': {'precision': 0.55052790346908,
   'recall': 0.05703125,
   'f1-score': 0.1033555146538298,
   'support': 6400},
  'accuracy': 0.80209375,
  'macro avg': {'precision': 0.6789720284489671,
   'recall': 0.5226953125,
   'f1-score': 0.4960636575324052,
   'support': 32000},
  'weighted avg': {'precision': 0.7560385034368993,
   'recall': 0.80209375,
   'f1-score': 0.7316885432595505,
   'support': 32000}}}

In [8]:
results['aloa_attack']['classification_report']

{'IN': {'precision': 0.8002075591711024,
  'recall': 0.9337109375,
  'f1-score': 0.8618196895675218,
  'support': 25600},
 'OUT': {'precision': 0.20291216533583842,
  'recall': 0.0675,
  'f1-score': 0.101301442138586,
  'support': 6400},
 'accuracy': 0.76046875,
 'macro avg': {'precision': 0.5015598622534704,
  'recall': 0.50060546875,
  'f1-score': 0.48156056585305385,
  'support': 32000},
 'weighted avg': {'precision': 0.6807484804040496,
  'recall': 0.76046875,
  'f1-score': 0.7097160400817346,
  'support': 32000}}

In [9]:
aloa.attack_model.threshold

0.995